In [1]:
pip install langgraph langchain gradio torch torchaudio demucs librosa torchcodec


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.9 MB/s eta 0:00:00
  Created wheel for demucs: filename=demucs-4.0.1-py3-none-any.whl size=78388 sha256=ead2991607e3132ec5821f987527b20c2e77cbc7ac5b45aa201d4899ba5df6e6
  Stored in directory: /root/.cache/pip/wheels/1b/0c/20/a3b3daa1f9b65c8b0445729f

In [2]:
from typing import TypedDict

class AudioState(TypedDict):
    file_path: str
    duration: float
    model_name: str
    vocals_path: str
    instrumental_path: str

AGENT 1: AUDIO ANALYZER

In [3]:
import librosa

def audio_analyzer(state):
    y, sr = librosa.load(state["file_path"], sr=None)
    duration = librosa.get_duration(y=y, sr=sr)

    return {
        **state,
        "duration": duration
    }

AGENT 2: MODEL SELECTION

In [4]:
def model_decider(state):
    if state["duration"] < 600:
        model = "htdemucs"
    else:
        model = "mdx"

    return {
        **state,
        "model_name": model
    }

AGENT 3: AUDIO SEPERATION


In [5]:
import subprocess
import os

def separator_agent(state):
    input_path = state["file_path"]
    model = state["model_name"]

    print(f"Running demucs with model: {model} and input: {input_path}")

    os.makedirs("outputs", exist_ok=True)

    try:
        result = subprocess.run(
            ["demucs", "-n", model, "-o", "outputs", input_path],
            check=True,
            capture_output=True,
            text=True
        )
        print("Demucs finished successfully.")
        print("Demucs stdout:", result.stdout)
        print("Demucs stderr:", result.stderr)

    except subprocess.CalledProcessError as e:
        print("Demucs failed")
        print("stdout:", e.stdout)
        print("stderr:", e.stderr)
        raise


    model_dir = os.path.join("outputs", model)
    if not os.path.exists(model_dir):
        raise FileNotFoundError(f"Model output directory not found: {model_dir}")


    subdirs = os.listdir(model_dir)
    if not subdirs:
        raise FileNotFoundError(f"No output folders found in {model_dir}")


    out_dir = max(
        [os.path.join(model_dir, d) for d in subdirs],
        key=os.path.getmtime
    )

    print("Detected Demucs output directory:", out_dir)
    print("Files inside:", os.listdir(out_dir))

    vocals_file = os.path.join(out_dir, "vocals.wav")


    possible_instrumentals = [
        "no_vocals.wav",
        "instrumental.wav",
        "other.wav"
    ]

    instrumental_file = None
    for fname in possible_instrumentals:
        path = os.path.join(out_dir, fname)
        if os.path.exists(path):
            instrumental_file = path
            break

    if not os.path.exists(vocals_file):
        raise FileNotFoundError(f"Vocals file missing: {vocals_file}")

    if instrumental_file is None:
        raise FileNotFoundError(
            f"No instrumental stem found in {out_dir}"
        )

    print("✔ Vocals file:", vocals_file)
    print("✔ Instrumental file:", instrumental_file)

    return {
        **state,
        "vocals_path": vocals_file,
        "instrumental_path": instrumental_file
    }


AGENT 4: POST SEPERATION

In [6]:
import subprocess
import os

def postprocess_agent(state):
    os.makedirs("outputs", exist_ok=True)

    vocals_out = "outputs/vocals_final.wav"
    inst_out = "outputs/instrumental_final.wav"

    print("Running ffmpeg for vocals...")
    try:
        result_vocals = subprocess.run(
            [
                "ffmpeg", "-y",
                "-i", state["vocals_path"],
                "-af", "loudnorm",
                "-ar", "44100",
                "-ac", "2",
                vocals_out
            ],
            check=True,
            capture_output=True,
            text=True
        )
        print("FFmpeg (vocals) stderr:\n", result_vocals.stderr)

    except subprocess.CalledProcessError as e:
        print("FFmpeg vocals failed")
        print(e.stderr)
        raise

    print("Running ffmpeg for instrumental...")
    try:
        result_inst = subprocess.run(
            [
                "ffmpeg", "-y",
                "-i", state["instrumental_path"],
                "-af", "loudnorm",
                "-ar", "44100",
                "-ac", "2",
                inst_out
            ],
            check=True,
            capture_output=True,
            text=True
        )
        print("FFmpeg (instrumental) stderr:\n", result_inst.stderr)

    except subprocess.CalledProcessError as e:
        print("FFmpeg instrumental failed")
        print(e.stderr)
        raise

    return {
        **state,
        "vocals_path": vocals_out,
        "instrumental_path": inst_out
    }


In [7]:
from langgraph.graph import StateGraph, END

def build_graph():
    graph = StateGraph(AudioState)

    graph.add_node("analyze", audio_analyzer)
    graph.add_node("decide_model", model_decider)
    graph.add_node("separate", separator_agent)
    graph.add_node("postprocess", postprocess_agent)

    graph.set_entry_point("analyze")

    graph.add_edge("analyze", "decide_model")
    graph.add_edge("decide_model", "separate")
    graph.add_edge("separate", "postprocess")
    graph.add_edge("postprocess", END)

    return graph.compile()

In [ ]:
import gradio as gr
import shutil
import uuid
import asyncio
import os

workflow = build_graph()

async def process_audio(audio):
    job_id = str(uuid.uuid4())
    input_path = f"uploads/{job_id}.wav"

    os.makedirs('uploads', exist_ok=True)
    shutil.copy(audio, input_path)


    final_state = await asyncio.to_thread(workflow.invoke, {
        "file_path": input_path
    })

    return final_state["vocals_path"], final_state["instrumental_path"]

demo = gr.Interface(
    fn=process_audio,
    inputs=gr.Audio(type="filepath", label="Upload Song"),
    outputs=[
        gr.Audio(label="Vocals"),
        gr.Audio(label="Instrumental")
    ],
    title="🎵 Music Separator",

)

demo.queue()
demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c79165eba6f9e3c58f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Running demucs with model: htdemucs and input: uploads/ac55eac3-02bd-4ae7-b426-e15aa4bd9b02.wav
Demucs finished successfully.
Demucs stdout: Downloading: "https://dl.fbaipublicfiles.com/demucs/hybrid_transformer/955717e8-8726e21a.th" to /root/.cache/torch/hub/checkpoints/955717e8-8726e21a.th
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/outputs/htdemucs
Separating track uploads/ac55eac3-02bd-4ae7-b426-e15aa4bd9b02.wav

Demucs stderr: 
  0%|          | 0.00/80.2M [00:00<?, ?B/s]
 16%|█▌        | 12.8M/80.2M [00:00<00:00, 133MB/s]
 41%|████      | 32.6M/80.2M [00:00<00:00, 177MB/s]
 65%|██████▌   | 52.4M/80.2M [00:00<00:00, 190MB/s]
 90%|████████▉ | 72.1M/80.2M [00:00<00:00, 196MB/s]
100%|██████████| 80.2M/80.2M [00:00<00:00, 191MB/s]

  0%|                                                                     | 0.0/35.099999999999994 [00:00<?, ?seconds/s]
 17%|██████████                                     